In [1]:
from naiveModel import NBLangIDModel
import pandas as pd
from sklearn.model_selection import train_test_split


/var/folders/6v/npbkg9_919s0m2j5qd702gf00000gn/T/ipykernel_20915/2055365162.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
naiveBayes = NBLangIDModel()


In [4]:
# load data, train test split
descriptions = pd.read_csv("cleanedData.csv")

print("Shape before dropping NaN values:", descriptions.shape)
descriptions = descriptions.dropna()
print("Shape after dropping NaN values:", descriptions.shape)

#TESTING THIS
# print("Rows to be dropped:")
# print(descriptions[descriptions['description'].isna() | ~descriptions['description'].apply(lambda x: isinstance(x, str))])
# #drop the 511 rows where description column is NaN or not a string
# descriptions = descriptions.dropna(subset=['description'])
# descriptions = descriptions[descriptions['description'].apply(lambda x: isinstance(x, str))]

train, test = train_test_split(descriptions, test_size=0.2)

Shape before dropping NaN values: (42661, 5)
Shape after dropping NaN values: (37478, 5)


In [5]:
train.drop("Unnamed: 0", axis= 1)
test.drop("Unnamed: 0", axis= 1)

,description,genre1,genre2,genre3
6202,The WINGS OF FIRE saga continues!Deep in the r...,'Fantasy','Dragons','Middle Grade'
28603,A summertime story only Elin Hilderbrand can t...,'Fiction','Chick Lit','Romance'
9532,The Wild Ass's Skin is Honoré de Balzac's 1831...,'Classics','France','Fiction'
39280,"Astutely observed and deftly witty, One Perfec...",'Nonfiction','Sociology','Marriage'
16179,A murder story with a devilish twist.Nick want...,'Horror','Young Adult','Fantasy'
...,...,...,...,...
3733,Dead Wrong is a study of the scientific and fo...,'Nonfiction','True Crime','History'
24405,"""I have written about the joys of love. I have...",'Historical Fiction','Fiction','Historical'
38663,Caldecott Honor BookWhat happens when Sal and ...,'Picture Books','Childrens','Fiction'
21685,Terra Cotta Warriors have been discovered on o...,'Science Fiction','Young Adult','Fantasy'


In [6]:
train_X = train['description']
train_y1 = train['genre1']
train_y2 = train['genre2']
train_y3 = train['genre3']

In [7]:
# fit the NB model 
naiveBayes.fit(train_X.tolist(), train_y1.tolist())